Batch Experiment Analysis
====

In [ ]:
import os
from glob import glob
import re
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from tqdm import tqdm
import random
import json

from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
from pprint import pprint

import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib
matplotlib.rcParams['figure.dpi'] = 120
matplotlib.rcParams['font.family'] = "serif"

from IPython.core.display import display, HTML

In [ ]:
from pathlib import Path
git_root_dir = !git rev-parse --show-toplevel
git_root_dir = Path(git_root_dir[0].strip())
git_root_dir

In [ ]:
import sys
sys.path.append(os.path.join(git_root_dir, 'src'))

In [ ]:
import cbrec.modeling
import cbrec.modeling.modelconfig
import cbrec.modeling.scorer
import cbrec.modeling.manager

In [ ]:
experiment_dir = '/home/lana/shared/caringbridge/data/projects/recsys-peer-match/torch_experiments/modeling/quantile_buckets_20211213214504'
assert os.path.exists(experiment_dir)

In [ ]:
!ls {experiment_dir}

In [ ]:
# glob(experiment_dir + '/outputs/*.json')
outputs = []
for fname in tqdm(glob(experiment_dir + '/../hyperparameter_*.json')):
    with open(fname) as infile:
        mm = cbrec.modeling.manager.ModelManager.load_from_filepath(fname)
        mm.load_model(load_preprocessor=False, load_model_state_dict=False, load_training_metrics=True)
        outputs.append(mm)

In [ ]:
losses = []
for mm in outputs:
    train_metrics, test_metrics = mm.model_trainer.get_train_metrics()
    best_loss = np.min(test_metrics.T[:,1])
    losses.append(best_loss)
    
fig, ax = plt.subplots(1, 1, figsize=(4,4))

ax.hist(losses, bins=20)
ax.set_title("Distribution of loss for all conditions")
ax.set_ylabel("Number of models")
ax.set_xlabel("Minimum loss achieved")

plt.show()

In [ ]:
# TODO loop through, do some stuff
mm.model_config